In [3]:
import pandas as pd
from numpy import *
from sklearn import *
random.seed(20200108)

## Read Data

In [10]:
file_path = 'Updated-CSV/specification'

In [13]:
data_hktv = pd.read_csv(f"{file_path}/HKTVMALL_specification.csv")
data_hktv = data_hktv.drop(data_hktv.columns[0], axis=1)
data_suning = pd.read_csv(f"{file_path}/Suning_specification.csv")
data_suning = data_suning.drop(data_suning.columns[0], axis=1)

In [17]:
data_suning

,Product Index,Product Name,Product Specification,Product Brand Number,Product Price
0,1.058485e+10,APPLE IPHONE XS 64GB 手機 金色,64GB,1501,7409.0
1,1.067406e+10,HUAWEI MATE 20 X 手機 藍色,None,1351,4988.0
2,1.058485e+10,APPLE IPHONE XR 128GB 手機 黑色,128GB,1501,5699.0
3,1.058485e+10,APPLE IPHONE XS MAX 256GB 手機 金色,256GB,1501,9199.0
4,1.055580e+10,Apple iPad Pro 11 256GB Wi-Fi Space Gray MTXQ2...,256GB,1451,6799.0
...,...,...,...,...,...
331,1.135650e+10,APPLE IPHONE 11 64GB 手機 白色,64GB,1501,5999.0
332,1.135651e+10,APPLE IPHONE 11 64GB 手機 綠色,64GB,1501,5999.0
333,1.136269e+10,HUAWEI P30 PRO 8+256GB 手機 薰衣草,256GB,1351,5988.0
334,1.135651e+10,APPLE IPHONE 11 PRO 64GB 手機 太空灰,64GB,1501,8599.0


## Price Factors

In [15]:
overall_df = pd.concat([data_hktv, data_suning])
overall_df.shape

(1339, 5)

In [19]:
Y_price = overall_df["Product Price"]

In [20]:
X_price = overall_df.drop(
    columns=["Product Index", "Product Name"]
)
X_price.shape

(1339, 3)

### Decision Tree Regressor

In [21]:
dtr_price = tree.DecisionTreeRegressor(max_depth=8)

In [ ]:
dtr_price.fit(X_price, Y_price)

In [ ]:
dtr_price.feature_importances_

## Sales Factors